In [5]:
import re
import plotly.express as px
import pandas as pd
import statsmodels
# pd.set_option("display.max_rows", None, "display.max_columns", None)

output_directory = 'output-sp2-2023-05-11'

def added_weekly(fig, idx):
    results = px.get_trendline_results(fig)
    return results.iloc[idx]["px_fit_results"].params[1] * pd.to_timedelta(1, unit='d').total_seconds() * 7

def estimate(fig, idx, date=pd.Timestamp.now()):
    results = px.get_trendline_results(fig)
    return results.iloc[idx]["px_fit_results"].params[0] + results.iloc[idx]["px_fit_results"].params[1] * date.timestamp()

In [10]:
df = pd.read_csv(f'../{output_directory}/kconfigreader/output.csv')
#df['committer_date'] = df['committer_date_unix'].apply(lambda d: pd.to_datetime(d, unit='s'))
#df = df.drop(columns=['kconfig-file', 'environment', 'binding-file', 'model-file', 'dimacs-file', 'dimacs-transformer', 'dimacs-time', 'model-time', 'smt-file', 'smt-time', 'smt-transformer', 'dimacs-analyzer', 'system', 'name', 'committer_date_readable', 'committer_date_unix'])
#df['model-count-log10'] = (df['model-count']).fillna(0).map(lambda s: len(str(s))) #todo
df

,system,revision,architecture,binding-file,kconfig-file,environment,model-file,model-features,model-variables,model-literals,model-time
0,linux,v2.5.45,alpha,kconfig-bindings/linux/v2.6.9.dumpconf,arch/alpha/Kconfig,SUBARCH=alpha|ARCH=alpha|SRCARCH=alpha|srctree...,linux/v2.5.45[alpha].model,3078.0,3439.0,62431.0,5314000269
1,linux,v2.5.45,arm,kconfig-bindings/linux/v2.6.9.dumpconf,arch/arm/Kconfig,SUBARCH=arm|ARCH=arm|SRCARCH=arm|srctree=.|CC=...,linux/v2.5.45[arm].model,3177.0,3521.0,75297.0,5592131793
2,linux,v2.5.45,cris,kconfig-bindings/linux/v2.6.9.dumpconf,arch/cris/Kconfig,SUBARCH=cris|ARCH=cris|SRCARCH=cris|srctree=.|...,linux/v2.5.45[cris].model,2947.0,3535.0,46920.0,5168454927
3,linux,v2.5.45,i386,kconfig-bindings/linux/v2.6.9.dumpconf,arch/i386/Kconfig,SUBARCH=i386|ARCH=i386|SRCARCH=i386|srctree=.|...,linux/v2.5.45[i386].model,3263.0,3629.0,66571.0,5469933187
4,linux,v2.5.45,ia64,kconfig-bindings/linux/v2.6.9.dumpconf,arch/ia64/Kconfig,SUBARCH=ia64|ARCH=ia64|SRCARCH=ia64|srctree=.|...,linux/v2.5.45[ia64].model,3090.0,3450.0,73935.0,5570946585
...,...,...,...,...,...,...,...,...,...,...,...
3141,linux,v6.3,s390,kconfig-bindings/linux/v6.3.dumpconf,Kconfig,SUBARCH=s390|ARCH=s390|SRCARCH=s390|srctree=.|...,linux/v6.3[s390].model,26074.0,27453.0,1421242.0,36270203944
3142,linux,v6.3,sh,kconfig-bindings/linux/v6.3.dumpconf,Kconfig,SUBARCH=sh|ARCH=sh|SRCARCH=sh|srctree=.|CC=cc|...,linux/v6.3[sh].model,26357.0,27803.0,1436333.0,37388843192
3143,linux,v6.3,sparc,kconfig-bindings/linux/v6.3.dumpconf,Kconfig,SUBARCH=sparc|ARCH=sparc|SRCARCH=sparc|srctree...,linux/v6.3[sparc].model,26169.0,27560.0,1441677.0,37055376977
3144,linux,v6.3,x86,kconfig-bindings/linux/v6.3.dumpconf,Kconfig,SUBARCH=x86|ARCH=x86|SRCARCH=x86|srctree=.|CC=...,linux/v6.3[x86].model,26639.0,28170.0,1593603.0,40886125893


In [227]:
fig = px.scatter(df, x='committer_date', y='source_lines_of_code', hover_data=['revision'], trendline='ols')
print("~#SLOC added per week: " + str(int(added_weekly(fig, 0))))
fig

~#SLOC added per week: 18660


In [135]:
#fig = px.scatter(df, x='committer_date', y='model-features', hover_data=['revision'], trendline='ols', color='extractor')
fig = px.box(df, x='revision', y='model-features', hover_data=['revision', 'architecture'])
# print("~#features added per week (kconfigreader): " + str(int(added_weekly(fig, 0))))
# print("~#features added per week (kmax): " + str(int(added_weekly(fig, 1))))
fig

In [187]:
import glob
import re

def inspect_architecture_features_for_revision(extractor, revision):
    architectures = [re.search('\[(.*)\]', f).group(1) for f in glob.glob(f'../{output_directory}/{extractor}/linux/{revision}[*.features')]
    architectures = list(set(architectures))
    architectures.sort()
    data = []
    for architecture in architectures:
        features_filename = f'../{output_directory}/{extractor}/linux/{revision}[{architecture}].features'
        with open(features_filename, 'r') as f:
            features = set([f.strip() for f in f.readlines()])
        data.append({'extractor': extractor, 'revision': revision, 'architecture': architecture, 'features': features, '#features': len(features)})
    common_features = set.intersection(*[d['features'] for d in data])
    for architecture in architectures:
        descriptor = [d for d in data if d['architecture'] == architecture][0]
        other_features = set.union(*[d['features'] for d in data if d['architecture'] != architecture])
        descriptor['#common_features'] = len(common_features)
        descriptor['owned_features'] = descriptor['features'].difference(other_features)
        descriptor['#owned_features'] = len(descriptor['owned_features'])
        descriptor['#shared_features'] = len(descriptor['features'].difference(common_features).difference(descriptor['owned_features']))
    return data

def inspect_architecture_features(extractor):
    revisions = [re.search('linux/(.*)\[', f).group(1) for f in glob.glob(f'../{output_directory}/{extractor}/linux/*.features')]
    revisions = list(set(revisions))
    revisions.sort()
    data = []
    for revision in revisions:
        print(revision)
        data += inspect_architecture_features_for_revision(extractor, revision)
    return pd.DataFrame(data).drop(columns=['features', 'owned_features'])

arch_features = inspect_architecture_features('kconfigreader')

v2.5.45
v2.5.46
v2.5.47
v2.5.48
v2.5.49
v2.5.50
v2.5.51
v2.5.52
v2.5.53
v2.5.54
v2.5.55
v2.5.56
v2.5.57
v2.5.58
v2.5.59
v2.5.60
v2.5.61
v2.5.62
v2.5.63
v2.5.64
v2.5.65
v2.5.66
v2.5.67
v2.5.68
v2.5.69
v2.5.70
v2.5.71
v2.5.72
v2.5.73
v2.5.74
v2.5.75
v2.6.0
v2.6.1
v2.6.10
v2.6.11
v2.6.12
v2.6.13
v2.6.14
v2.6.15
v2.6.16
v2.6.17
v2.6.18
v2.6.19
v2.6.2
v2.6.20
v2.6.21
v2.6.22
v2.6.23
v2.6.24
v2.6.25
v2.6.26
v2.6.27
v2.6.28
v2.6.29
v2.6.3
v2.6.30
v2.6.31
v2.6.32
v2.6.33
v2.6.34
v2.6.35
v2.6.36
v2.6.37
v2.6.38
v2.6.39
v2.6.4
v2.6.5
v2.6.6
v2.6.7
v2.6.8
v2.6.9
v3.0
v3.1
v3.10
v3.11
v3.12
v3.13
v3.14
v3.15
v3.16
v3.17
v3.18
v3.19
v3.2
v3.3
v3.4
v3.5
v3.6
v3.7
v3.8
v3.9
v4.0
v4.1
v4.10
v4.11
v4.12
v4.13
v4.14
v4.15
v4.16
v4.17
v4.18
v4.19
v4.2
v4.20
v4.3
v4.4
v4.5
v4.6
v4.7
v4.8
v4.9
v5.0
v5.1
v5.10
v5.11
v5.12
v5.13
v5.14
v5.15
v5.16
v5.17
v5.18
v5.19
v5.2
v5.3
v5.4
v5.5
v5.6
v5.7
v5.8
v5.9
v6.0
v6.1
v6.2
v6.3


In [224]:
arch2 = arch_features.melt(id_vars=['extractor', 'revision', 'architecture'], value_vars=['#features', '#common_features', '#owned_features', '#shared_features'])
fig = px.box(arch2, x='revision', y='value', color='variable')
fig
# todo: calculate core dead features and remove(?) them from here, or plot them, and find a "good" definition of what it means for an architecture to "include" a feature
#.e.g., dead feature occur in a formula, but not really in the architecture (related work does not do this)
# i guess there may be many core/dead features per arch, as later all arch's have been unified and decision propagation does the rest => i.e., it is important to analyze formulas, not only syntax of kconfig, so our computation effort is justified

In [219]:
arch2 = arch_features.melt(id_vars=['extractor', 'revision', 'architecture'], value_vars=['#common_features', '#shared_features', '#owned_features'])
fig = px.bar(arch2[arch2['architecture'] == 'x86'], x='revision', y='value', color='variable')
fig = px.bar(arch2, x='revision', y='value', color='variable', log_y=True)
fig

In [90]:
from distutils.version import StrictVersion
dff = df[['revision', 'model-features']].groupby('revision').max().reset_index()
dff['revision'] = df['revision'].map(lambda s: s[1:]).apply(StrictVersion)
fig = px.scatter(dff.sort_values(by='revision'), y='model-features')
fig

In [76]:
fig = px.scatter(df, x='committer_date', y='dimacs-variables', hover_data=['revision'], trendline='ols', color='extractor')
print("~#variables added per week (kconfigreader): " + str(int(added_weekly(fig, 0))))
print("~#variables added per week (kmax): " + str(int(added_weekly(fig, 1))))
fig

ValueError: Value of 'x' is not the name of a column in 'data_frame'. Expected one of ['system', 'revision', 'architecture', 'binding-file', 'kconfig-file', 'environment', 'model-file', 'model-features', 'model-variables', 'model-literals', 'model-time'] but received: committer_date

In [253]:
df2 = df[df['model-count-log10'] > 1]
fig = px.scatter(df2, x='committer_date', y='model-count-log10', hover_data=['revision'], color='extractor', trendline='ols')
print("~#configurations scaled per week (kconfigreader): " + str(int(pow(10, added_weekly(fig, 0)))))
print("~#configurations scaled per week (kmax): " + str(int(pow(10, added_weekly(fig, 1)))))
print("~#configurations today (kconfigreader): 10^" + str(int(estimate(fig, 0))))
print("~#configurations today (kmax): 10^" + str(int(estimate(fig, 1))))
fig

~#configurations scaled per week (kconfigreader): 37
~#configurations scaled per week (kmax): 12
~#configurations today (kconfigreader): 10^2322
~#configurations today (kmax): 10^1593


In [4]:
archdf = pd.read_csv(f'../{output_directory}/read-linux-architectures/output.csv')
archdf['committer_date_unix'] = archdf['committer_date_unix'].apply(lambda d: pd.to_datetime(d, unit='s'))
fig = px.line(archdf, x='committer_date_unix', y='architecture', hover_data=['revision'], color='architecture')
fig.update_layout(height=1000)
fig

In [194]:
df = pd.read_csv('../output/kconfig/output.csv')
df['committer_date_unix'] = df['committer_date_unix'].apply(lambda d: pd.to_datetime(d, unit='s'))

fig = px.scatter(df, x='model-features', y='revision-parameter', color='extractor', size='model-literals')
fig.show()
#todo: for history, consider boxplot

In [198]:
fig = px.box(df, x='committer_date_unix', y='model-features', color='extractor')
fig.show()
#todo: for history, consider boxplot